In [1]:
from rdkit import Chem
from datasets import FishTox

dataset = FishTox('/data')
smiles = dataset.smiles


/home/marco/anaconda3/envs/InfoMol/lib/python3.9/site-packages/torch_geometric/typing.py:47: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.29' not found (required by /home/marco/anaconda3/envs/InfoMol/lib/python3.9/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/home/marco/anaconda3/envs/InfoMol/lib/python3.9/site-packages/torch_geometric/typing.py:101: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.29' not found (required by /home/marco/anaconda3/envs/InfoMol/lib/python3.9/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "
/home/marco/anaconda3/envs/InfoMol/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io

In [4]:
from encoder import KPGTEncoder

encoder = KPGTEncoder(verbose=True)

/home/marco/anaconda3/envs/InfoMol/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/marco/anaconda3/envs/InfoMol/lib/python3.9/site-packages/torch_geometric/typing.py:47: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.29' not found (required by /home/marco/anaconda3/envs/InfoMol/lib/python3.9/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/home/marco/anaconda3/envs/InfoMol/lib/python3.9/site-packages/torch_geometric/typing.py:101: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.29' not found (required by /home/marco/anaconda3/envs/InfoMol/lib/python3.9/site-packages/li

Instantiating the model.
Instantiating the LiGhT transformer.
Loading the weights from /home/marco/Documents/InfoMol/KPGT/model/base.pth.


In [7]:
encoder(smiles)

100%|██████████| 10/10 [00:01<00:00,  7.95it/s]


array([[-1.0110782 ,  1.00954   , -1.8167145 , ..., -2.5774841 ,
         0.05610704, -0.29041302],
       [ 0.47611442, -1.0293853 , -0.8827675 , ..., -0.70951074,
        -2.9408808 ,  1.2040155 ],
       [ 0.8174975 , -1.886406  , -1.6213434 , ..., -0.853308  ,
        -2.485643  ,  0.63166   ],
       ...,
       [-2.5447195 ,  2.7243295 ,  0.8932501 , ...,  0.01272453,
        -3.8188992 ,  1.6454501 ],
       [-0.06776428,  3.4346304 , -0.1998607 , ...,  0.5340105 ,
        -3.8123994 ,  0.04437614],
       [-4.051842  , -1.1328802 , -0.46681842, ...,  0.799869  ,
        -4.9121065 ,  2.3213289 ]], dtype=float32)

In [1]:
import os
import sys
__file__='/home/marco/Documents/InfoMol/notebooks/KPGT.ipynb'

import scipy.stats

module_path = os.path.abspath(os.path.join('/'.join(__file__.split('/')[:-1]), '..', 'KPGT'))
sys.path.insert(0, module_path)

from src.model.light import LiGhTPredictor as LiGhT
from src.model_config import config_dict
from src.data.finetune_dataset import MoleculeDataset
from src.data.featurizer import Vocab, N_ATOM_TYPES, N_BOND_TYPES
import torch


# Model Initialization
vocab = Vocab(N_ATOM_TYPES, N_BOND_TYPES)
config = config_dict['base']
model_path = '../model/base.pth'
model = LiGhT(
    d_node_feats=config['d_node_feats'],
    d_edge_feats=config['d_edge_feats'],
    d_g_feats=config['d_g_feats'],
    d_hpath_ratio=config['d_hpath_ratio'],
    n_mol_layers=config['n_mol_layers'],
    path_length=config['path_length'],
    n_heads=config['n_heads'],
    n_ffn_dense_layers=config['n_ffn_dense_layers'],
    input_drop=0,
    attn_drop=0,
    feat_drop=0,
    n_node_types=vocab.vocab_size
    )
model.load_state_dict({k.replace('module.',''):v for k,v in torch.load(model_path).items()})


<All keys matched successfully>

In [2]:
import numpy as np
from src.data.descriptors.rdNormalizedDescriptors import RDKit2DNormalized
from src.data.featurizer import smiles_to_graph_tune


smile = smiles[0]

generator = RDKit2DNormalized()
md = torch.Tensor(generator.process(smile)[1:]).reshape(1,-1)

mol = Chem.MolFromSmiles(smile)
fp = torch.Tensor(Chem.RDKFingerprint(mol, minPath=1, maxPath=7, fpSize=512)).reshape(1,-1)

graph = smiles_to_graph_tune(
    smile,
    max_length=config['path_length'],
    n_virtual_nodes=2
) 
model.eval()

rep = model.generate_fps(graph, fp, md)
rep.shape

NameError: name 'smiles' is not defined

In [ ]:
from src.data.collator import Collator_tune

collator = Collator_tune(config['path_length'])

In [ ]:
import dgl
batched_graph = dgl.batch(graphs)

In [ ]:
scipy.stats.gilbrat

In [ ]:
fps = model.generate_fps(graphs[0], fp[0], md[0])

In [ ]:
fps.shape

In [ ]:
config['d_g_feats']*3